# Test the `map()` Function to Format Datasets

## Install and Import Dependencies

In [1]:
!pip install -qU datasets pprint

ERROR: Could not find a version that satisfies the requirement pprint (from versions: none)
ERROR: No matching distribution found for pprint


In [2]:
from datasets import load_dataset
import pprint
import torch

## Load Dataset

In [3]:
dataset = load_dataset("claudios/code_search_net", "go", split="train[:10]")

Generating train split:   0%|          | 0/317832 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/14291 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/14242 [00:00<?, ? examples/s]

In [4]:
print(dataset)

Dataset({
    features: ['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_documentation_string', 'func_code_url'],
    num_rows: 10
})


## Define Prompt Format Function for a Single Sample

In [18]:
# EOS Token is required to stop open-ended generation and eventual hallucination
# EOS_TOKEN = tokenizer.eos_token
EOS_TOKEN = "<|end_of_text|>"

In [6]:
def formatFunctionSample(sample):
    language = sample['language']
    instruction = f"What does this {language} function do?"
    inputText = sample['func_code_string']
    outputText = sample['func_documentation_string']

    return {
        "instruction": instruction,
        "input": inputText,
        "output": outputText
    }

## Use the `map()` Method to Format the Dataset

In [8]:
formattedDataset = dataset.map(formatFunctionSample, remove_columns=dataset.column_names)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## Sample Formatted Dataset

In [9]:
len(formattedDataset)

10

In [10]:
pprint.pp(formattedDataset[1])

{'instruction': 'What does this go function do?',
 'input': 'func (s *UpdateSkillGroupInput) SetDescription(v string) '
          '*UpdateSkillGroupInput {\n'
          '\ts.Description = &v\n'
          '\treturn s\n'
          '}',
 'output': "// SetDescription sets the Description field's value."}


## Create Dataset of *Alpaca* Formatted Samples

In [13]:
sample = formattedDataset[2]

In [19]:
alpacaFormatString = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{sample['instruction']}

### Input:
{sample['input']}

### Response:
{sample['output']}"""

alpacaFormatString += EOS_TOKEN

In [20]:
print(alpacaFormatString)

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
What does this go function do?

### Input:
func (s *UpdateSkillGroupInput) SetSkillGroupArn(v string) *UpdateSkillGroupInput {
	s.SkillGroupArn = &v
	return s
}

### Response:
// SetSkillGroupArn sets the SkillGroupArn field's value.<|end_of_text|>
